# Eligibility for mobilization: Cohort identification. 

This script identifies the cohort using CLIF 2.0 tables. 

Requirements:
* Required table filenames should be clif_patient, clif_hospitalization, clif_adt, clif_vitals, clif_labs, clif_medication_admin_continuous, clif_respiratory_support

## Load Libraries

In [ ]:
import sys
import os
import time
import pandas as pd
import numpy as np
import duckdb
import pyCLIF

import seaborn as sns
import matplotlib.pyplot as plt

## Required columns and categories

In [2]:
rst_required_columns = [
    'hospitalization_id',
    'recorded_dttm',
    'device_name',
    'device_category',
    'mode_name', 
    'mode_category',
    'tracheostomy',
    'fio2_set',
    'lpm_set',
    'resp_rate_set',
    'peep_set',
    'resp_rate_obs',
    'tidal_volume_set'
]

vitals_required_columns = [
    'hospitalization_id',
    'recorded_dttm',
    'vital_category',
    'vital_value'
]
vitals_of_interest = ['heart_rate', 'resp_rate', 'sbp', 'dbp', 'map', 'resp_rate', 'spo2']

labs_required_columns = [
    'hospitalization_id',
    'lab_result_dttm',
    'lab_category',
    'lab_value'
]
labs_of_interest = ['lactate']

meds_required_columns = [
    'hospitalization_id',
    'admin_dttm',
    'med_name',
    'med_category',
    'med_dose',
    'med_dose_unit'
]
meds_of_interest = [
    'norepinephrine', 'epinephrine', 'phenylephrine', 'vasopressin',
    'dopamine', 'angiotensin', 'nicardipine', 'nitroprusside',
    'clevidipine', 'cisatracurium'
]

## Load data

In [ ]:
patient = pyCLIF.load_data('clif_patient')
hospitalization = pyCLIF.load_data('clif_hospitalization')

In [4]:
# Standardize all _dttm variables to the same format
patient = pyCLIF.standardize_datetime(patient)
hospitalization = pyCLIF.standardize_datetime(hospitalization)

In [ ]:
patient = pyCLIF.remove_duplicates(patient, ['patient_id'], 'patient')
hospitalization = pyCLIF.remove_duplicates(hospitalization, ['hospitalization_id'], 'hospitalization')

In [ ]:
print(f"Total Number of unique encounters in the data: {pyCLIF.count_unique_encounters(hospitalization, 'hospitalization_id')}")

## Cohort Identification

### Inclusion Criteria:

* Filter Admissions for March 1, 2020 - March 31, 2022
* Encounters receiving invasive mechanical ventilation during this period

### Exclusion criteria:

1. Encounters that were on vent for less than 2 hours
2. Encounters that were on trach in the first 72 hours 
3. Encounters that received Cisatracurium for 4 hours or more

In [ ]:
cohort = hospitalization[
    (hospitalization['admission_dttm'] >= '2020-03-01') &
    (hospitalization['admission_dttm'] <= '2022-03-31') &
    (hospitalization['age_at_admission'] >= 18)
].reset_index(drop=True)[['hospitalization_id']].drop_duplicates()

cohort_ids = cohort['hospitalization_id'].unique().tolist()
print(f"Number of unique encounters after filtering by date and age:", cohort['hospitalization_id'].nunique())

In [ ]:
# Import clif respiratory table for this cohort
rst_filters = {
    'hospitalization_id': cohort_ids
}
resp_support_raw = pyCLIF.load_data('clif_respiratory_support', columns=rst_required_columns, filters=rst_filters)
resp_support = resp_support_raw.copy()
resp_support['device_category'] = resp_support['device_category'].str.lower()
resp_support['mode_category'] = resp_support['mode_category'].str.lower()

In [ ]:
# Apply Nick's Waterfall fill logic for respiratory support table
processed_resp_support = pyCLIF.process_resp_support(resp_support)

In [ ]:
# Identify the cohort on invasive mechanical ventilation 
columns_to_keep = [
    'hospitalization_id', 'recorded_dttm', 'device_name','device_category',
    'mode_name', 'mode_category' , 'tracheostomy',
    'fio2_set', 'lpm_set', 'peep_set', 
    'resp_rate_obs', 'resp_rate_set'
]

ventilator_usage = processed_resp_support[processed_resp_support['device_category'].str.contains("imv", case=False, na=False)]
cohort_on_vent = ventilator_usage.merge(cohort, on='hospitalization_id', how='left')
cohort_ids = cohort_on_vent['hospitalization_id'].unique().tolist()
cohort_on_vent.loc[:, 'recorded_dttm'] = pd.to_datetime(cohort_on_vent['recorded_dttm'])

cohort_on_vent = cohort_on_vent[columns_to_keep]
cohort_on_vent['on_vent'] = cohort_on_vent['device_category'].str.contains("imv", case=False, na=False).astype(int)
cohort_on_vent = cohort_on_vent.sort_values(by=['hospitalization_id', 'recorded_dttm'])

vent_start_end = cohort_on_vent[cohort_on_vent['on_vent'] == 1].groupby('hospitalization_id').agg(
    vent_start_time=('recorded_dttm', 'min'),
    vent_end_time=('recorded_dttm', 'max'),
    recorded_dttm=('recorded_dttm', lambda x: x)
).reset_index()

print(f"Number of unique encounters after filtering for ventilator usage: {cohort_on_vent['hospitalization_id'].nunique()}")

In [ ]:
# Import clif continuous meds for the cohort on vent during the required time period
meds_filters = {
    'hospitalization_id': cohort_ids,
    'med_category': meds_of_interest
}
meds = pyCLIF.load_data('clif_medication_admin_continuous', columns=meds_required_columns, filters=meds_filters)
print("unique encounters in meds", pyCLIF.count_unique_encounters(meds))

In [ ]:
# Import clif continuous meds and clif labs table for the cohort on vent during the required time period
labs_filters = {
    'hospitalization_id': cohort_ids,
    'lab_category': labs_of_interest
}
labs = pyCLIF.load_data('clif_labs', columns=labs_required_columns, filters=labs_filters)
print("unique encounters in labs", pyCLIF.count_unique_encounters(labs))

In [ ]:
vitals_filters = {
    'hospitalization_id': cohort_ids,
    'vital_category': vitals_of_interest
}
vitals = pyCLIF.load_data('clif_vitals', columns=vitals_required_columns, filters=vitals_filters)


In [ ]:
# Get first_vital_dttm and last_vital_dttm for each hospitalization_id in one step
vital_dttm_bounds = vitals.groupby('hospitalization_id')['recorded_dttm'].agg(['min', 'max']).reset_index()
vital_dttm_bounds.columns = ['hospitalization_id', 'first_vital_dttm', 'last_vital_dttm']
print("unique encounters in vitals", pyCLIF.count_unique_encounters(vital_dttm_bounds))

## Hourly sequence for the cohort

In [ ]:
final_cohort = vent_start_end.merge(vital_dttm_bounds, on='hospitalization_id', how='inner')
print("unique encounters in resp filtered", pyCLIF.count_unique_encounters(final_cohort))

In [ ]:
# Function to generate hourly sequence for each group (hospitalization_id)
def generate_hourly_sequence(group):
    # Get the vent start time and discharge time
    start_time = group['vent_start_time'].iloc[0]
    last_vital_time = group['last_vital_dttm'].iloc[0]
    
    # Check if the last_vital_time is before the vent start time
    if last_vital_time < start_time:
        end_time = group['recorded_dttm'].max()  # Use the max recorded_dttm in this case
    else:
        end_time = last_vital_time  # Otherwise, use the last_vital_time
    
    # Generate the sequence of hourly timestamps
    hourly_timestamps = pd.date_range(start=start_time, end=end_time, freq='h')
    
    # Create a new DataFrame for this sequence
    return pd.DataFrame({
        'hospitalization_id': group['hospitalization_id'].iloc[0],
        'recorded_dttm': hourly_timestamps
    })

# Apply the function to each group and concatenate the results, using include_groups=False
# hour_sequence = final_cohort.groupby('hospitalization_id').apply(generate_hourly_sequence).reset_index(drop=True)
hour_sequence = final_cohort.groupby('hospitalization_id')\
    .apply(generate_hourly_sequence)\
    .reset_index(drop=True)

# Add `recorded_date` and `recorded_hour` columns
# Convert recorded_dttm to datetime if not already done
hour_sequence['recorded_dttm'] = pd.to_datetime(hour_sequence['recorded_dttm'])
hour_sequence['recorded_date'] = hour_sequence['recorded_dttm'].dt.date
hour_sequence['recorded_hour'] = hour_sequence['recorded_dttm'].dt.hour
hour_sequence['time_from_vent'] = hour_sequence.groupby('hospitalization_id').cumcount()

## Hourly Respiratory support

In [17]:
resp_filtered = cohort_on_vent[columns_to_keep]
# Apply thresholds and replace values outside these with NaN using .loc[]
cohort_on_vent.loc[:, 'fio2_set'] = cohort_on_vent['fio2_set'].where(cohort_on_vent['fio2_set'].between(0.21, 1, inclusive='both'), np.nan)
cohort_on_vent.loc[:, 'resp_rate_set'] = cohort_on_vent['resp_rate_set'].where(cohort_on_vent['resp_rate_set'].between(0, 60, inclusive='both'), np.nan)
cohort_on_vent.loc[:, 'peep_set'] = cohort_on_vent['peep_set'].where(cohort_on_vent['peep_set'].between(0, 50, inclusive='both'), np.nan)
cohort_on_vent.loc[:, 'resp_rate_obs'] = cohort_on_vent['resp_rate_obs'].where(cohort_on_vent['resp_rate_obs'].between(0, 100, inclusive='both'), np.nan)
cohort_on_vent.loc[:, 'lpm_set'] = cohort_on_vent['lpm_set'].where(cohort_on_vent['lpm_set'].between(0, 60, inclusive='both'), np.nan)

cohort_on_vent['recorded_date'] = cohort_on_vent['recorded_dttm'].dt.date
cohort_on_vent['recorded_hour'] = cohort_on_vent['recorded_dttm'].dt.hour
hourly_vent_df = cohort_on_vent.groupby(['hospitalization_id', 'recorded_date', 'recorded_hour']).agg(
    min_resp_rate_obs=pd.NamedAgg(column='resp_rate_obs', aggfunc='min'),
    min_lpm_set=pd.NamedAgg(column='lpm_set', aggfunc='min'),
    min_fio2_set=pd.NamedAgg(column='fio2_set', aggfunc='min'),
    min_peep_set=pd.NamedAgg(column='peep_set', aggfunc='min'),
    max_resp_rate_obs=pd.NamedAgg(column='resp_rate_obs', aggfunc='max'),
    max_lpm_set=pd.NamedAgg(column='lpm_set', aggfunc='max'),
    max_fio2_set=pd.NamedAgg(column='fio2_set', aggfunc='max'),
    max_peep_set=pd.NamedAgg(column='peep_set', aggfunc='max'),
    hourly_trach=pd.NamedAgg(column='tracheostomy', aggfunc=lambda x: 1 if x.max() == 1 else 0),
    hourly_on_vent=pd.NamedAgg(column='on_vent', aggfunc=lambda x: 1 if x.max() == 1 else 0)
).reset_index()

In [ ]:
# Merge hourly_vent_df with hour_sequence on hospitalization_id, recorded_date, and recorded_hour
final_df = pd.merge(hour_sequence, hourly_vent_df, on=['hospitalization_id', 'recorded_date', 'recorded_hour'], 
                     how='left')
print("unique encounters in merged df", pyCLIF.count_unique_encounters(final_df))

## Hourly Vitals

In [23]:
vitals['vital_value'] = pd.to_numeric(vitals['vital_value'], errors='coerce')

vitals['recorded_dttm'] = pd.to_datetime(vitals['recorded_dttm'])
vitals['recorded_hour'] = vitals['recorded_dttm'].dt.hour
vitals['recorded_date'] = vitals['recorded_dttm'].dt.date

vitals_min_max = vitals.groupby(['hospitalization_id', 'recorded_date', 'recorded_hour', 'vital_category']).agg(
    min=pd.NamedAgg(column='vital_value', aggfunc='min'),
    max=pd.NamedAgg(column='vital_value', aggfunc='max')
).reset_index()

# Pivot the table to reshape it
vitals_pivot = vitals_min_max.pivot_table(
    index=['hospitalization_id', 'recorded_date', 'recorded_hour'],
    columns='vital_category',
    values=['min', 'max']
).reset_index()

# Flatten the column multi-index after pivot
vitals_pivot.columns = ['_'.join(col).strip() if type(col) is tuple else col for col in vitals_pivot.columns]
# Remove trailing underscores
vitals_pivot.columns = [col.rstrip('_') for col in vitals_pivot.columns]

In [ ]:
print(vitals_pivot.columns)

In [25]:
# merge vitals with the blocked resp support data
final_df = pd.merge(final_df, vitals_pivot, on=['hospitalization_id', 'recorded_date', 'recorded_hour'], 
                   how='outer')

In [ ]:
final_df.columns

In [ ]:
final_df.shape